In [1]:
# Agenda Based Search (2.0) 


# Importing Libraries
import pandas as pd
import math
import hashlib
import string
import heapq
import random


df = pd.read_csv("tubedata.csv", header=None)
df.head()
from collections import defaultdict
station_dict = defaultdict(list)
zone_dict = defaultdict(set)

for index, row in df.iterrows():
  start_station = row[0]
  end_station = row[1]
  act_cost = int(row[3])
  zone1 = row[4]
  zone2 = row[5]
  # station dictionary of child station tuples
  # (child_name, cost from parent to the child)
  # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
  station_list = station_dict[start_station]
  station_list.append((end_station, act_cost))
  # the following two lines add the other direction of the tube "step"
  station_list = station_dict[end_station]
  station_list.append((start_station, act_cost))
  # we add the main zone
  zone_dict[start_station].add(zone1)
  # we add the secondary zone
  if zone2 != "0":
    zone_dict[start_station].add(zone2)
    # if the secondary zone is not 0 it’s the main zone for the ending station
    zone_dict[end_station].add(zone2)
  else:
    # otherwise the main zone for the ending station
    # is the same as for the starting station
    zone_dict[end_station].add(zone1)


In [2]:
#BFS Search (2.1.1)

from collections import deque

#BFS function with station graph, start station and goal station as parameters
def bfs(graph, start, goal):
    visited = set()
    queue = deque([(start, [], 0, 0)])  # Add parameters for total cost and nodes expanded

    while queue:
        current_station, path, total_cost, nodes_expanded = queue.popleft()

        if current_station == goal:
            average_cost = total_cost if len(path) > 0 else 0
            return path + [current_station], average_cost, nodes_expanded

        if current_station not in visited:
            visited.add(current_station)
            for neighbor, cost in graph[current_station]:
                if neighbor not in visited:
                    queue.append((neighbor, path + [current_station], total_cost + cost, nodes_expanded + 1))

    
    return None, 0, 0


# Shows connections between stations and costs
graph = dict(station_dict)

start_station = "Tottenham Court Road"
end_station = "Stratford"

shortest_path, average_cost, nodes_expanded = bfs(graph, start_station, end_station)

if shortest_path:
    print(f"Shortest path from {start_station} to {end_station}: {shortest_path}")
    print(f"Average time : {average_cost}")
    print(f"Number of nodes expanded: {nodes_expanded}")
else:
    print(f"No path found from {start_station} to {end_station}")


Shortest path from Tottenham Court Road to Stratford: ['Tottenham Court Road', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Average time : 18
Number of nodes expanded: 8


In [3]:
#DFS Search (2.1.2)

#DFS function with station graph, start station and goal station as parameters
def dfs_path(graph, start, goal):
    visited = set()

    def dfs(current_station, path, total_cost, nodes_expanded):
        visited.add(current_station)

        if current_station == goal:
            average_cost = total_cost if len(path) > 0 else 0
            return path + [current_station], average_cost, nodes_expanded

        for neighbor, cost in graph[current_station]:
            if neighbor not in visited:
                res = dfs(neighbor, path + [current_station], total_cost + cost, nodes_expanded + 1)
                if res:
                    return res

        return None

    res = dfs(start, [], 0, 0)
    return res

# Shows connections between stations and costs
graph = dict(station_dict)

start_station = "Tottenham Court Road"
end_station = "Stratford"

dfs_res, average_time, nodes_expanded = dfs_path(graph, start_station, end_station)

if dfs_res:
    print(f"DFS path from {start_station} to {end_station}: {dfs_res}")
    print(f"Average time: {average_time}")
    print(f"Number of nodes expanded: {nodes_expanded}")
else:
    print(f"No path found from {start_station} to {end_station}")



DFS path from Tottenham Court Road to Stratford: ['Tottenham Court Road', 'Oxford Circus', "Regent's Park", 'Baker Street', 'Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush", 'White City', 'East Acton', 'North Acton', 'West Acton', 'Ealing Broadway', 'Ealing Common', 'Acton Town', 'Chiswick Park', 'Turnham Green', 'Stamford Brook', 'Ravenscourt Park', 'Hammersmith', 'Barons Court', 'West Kensington', "Earls' Court", 'High Street Kensington', 'Gloucester Road', 'South Kensington', 'Sloane Square', 'Victoria', "St. James' Park", 'Westminster', 'Embankment', 'Charing Cross', 'Piccadilly Circus', 'Leicester Square', 'Covent Garden', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Average time: 94
Number of nodes expanded: 45


In [4]:
#UCS Search (2.1.3)


import heapq

#UCS function with station graph, start station and goal station as parameters
def ucs_path(graph, start, goal):
    visited = set()
    priority_queue = [(0, start, [], 0)]  # (cost, current_station, path, nodes_expanded)

    while priority_queue:
        current_cost, current_station, path, nodes_expanded = heapq.heappop(priority_queue)

        if current_station == goal:
            average_time = current_cost if len(path) > 0 else 0
            return path + [current_station], average_time, nodes_expanded

        if current_station not in visited:
            visited.add(current_station)
            for neighbor, cost in graph[current_station]:
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (current_cost + cost, neighbor, path + [current_station], nodes_expanded + 1))

   
    return None, 0, 0

# Shows connections between stations and costs
graph = dict(station_dict)

start_station = "Tottenham Court Road"
end_station = "Stratford"

ucs_res, average_time, nodes_expanded = ucs_path(graph, start_station, end_station)

if ucs_res:
    print(f"UCS path from {start_station} to {end_station}: {ucs_res}")
    print(f"Average time: {average_time}")
    print(f"Number of nodes expanded: {nodes_expanded}")
else:
    print(f"No path found from {start_station} to {end_station}")


UCS path from Tottenham Court Road to Stratford: ['Tottenham Court Road', 'Holborn', 'Chancery Lane', "St. Paul's", 'Bank/Monument', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Average time: 18
Number of nodes expanded: 8


In [5]:
# Cost Function extension (2.3.0)
#UCS line change function with station graph, start station and goal station as parameters
def ucs_line_change(graph, start, goal, line_change_time=2):
    visited = set()
    priority_queue = [(0, start, [], 0)]  # (cost, current_station, path, total_time)

    while priority_queue:
        current_cost, current_station, path, total_time = heapq.heappop(priority_queue)

        if current_station == goal:
            average_time = total_time if len(path) > 0 else 0
            return path + [current_station], average_time

        if current_station not in visited:
            visited.add(current_station)
            for neighbor, cost, line_change in graph[current_station]:
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (current_cost + cost, neighbor, path + [current_station], total_time + cost + line_change_time * line_change))

    
    return None, 0


#  0 if no line change, 1 if line change is needed
graph = {
    "Tottenham Court Road": [("Holborn", 2, 0), ("Oxford Circus", 3, 1)],
    "Holborn": [("Tottenham Court Road", 2, 0), ("Russell Square", 1, 0)],
    "Oxford Circus": [("Tottenham Court Road", 3, 1), ("Bond Street", 2, 0)],
    "Russell Square": [("Holborn", 1, 0), ("Stratford", 5, 0)],
    "Bond Street": [("Oxford Circus", 2, 0)],
    "Stratford": [("Russell Square", 5, 0)]
}

start_station = "Tottenham Court Road"
end_station = "Stratford"

ucs_result, average_time = ucs_line_change(graph, start_station, end_station)

if ucs_result:
    print(f"UCS path from {start_station} to {end_station}: {ucs_result}")
    print(f"Average time: {average_time}")
else:
    print(f"No path found from {start_station} to {end_station}")


UCS path from Tottenham Court Road to Stratford: ['Tottenham Court Road', 'Holborn', 'Russell Square', 'Stratford']
Average time: 8


In [6]:
# Heuristic Search (2.4)
#BFS with heuristic function with station graph, start station, goal station and zones as parameters
def bfs_heuristic(graph, start, goal, zone_dict):
    visited = set()
    priority_queue = [(heuristic(start, goal, zone_dict), start, [], 0)]  # (heuristic, current_station, path, total_time)

    while priority_queue:
        _, current_station, path, total_time = heapq.heappop(priority_queue)

        if current_station == goal:
            average_time = total_time if len(path) > 0 else 0
            return path + [current_station], average_time

        if current_station not in visited:
            visited.add(current_station)
            for neighbor, cost in graph[current_station]:
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (heuristic(neighbor, goal, zone_dict), neighbor, path + [current_station], total_time + cost))

   
    return None, 0

def heuristic(station, goal, zone_dict):
    # A simple heuristic: the number of zones between the station and the goal
    if station in zone_dict and goal in zone_dict:
        return abs(len(zone_dict[station]) - len(zone_dict[goal]))
    else:
        return 0  # Default heuristic value if zone information is not available

# Shows connections between stations and costs
graph = dict(station_dict)

start_station = "Tottenham Court Road"
end_station = "Stratford"

bfs_result, average_time = bfs_heuristic(graph, start_station, end_station, zone_dict)

if bfs_result:
    print(f"BFS with Heuristic path from {start_station} to {end_station}: {bfs_result}")
    print(f"Average time: {average_time}")
else:
    print(f"No path found from {start_station} to {end_station}")


BFS with Heuristic path from Tottenham Court Road to Stratford: ['Tottenham Court Road', 'Oxford Circus', 'Bond Street', 'Baker Street', 'Great Portland Street', 'Euston Square', "King's Cross St. Pancras", 'Farringdon', 'Barbican', 'Moorgate', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Average time: 30


In [7]:
# EECS Username Password Generation (3.0.1)

def get_password(student_username, l=10):
# possible characters include upper-case English letters,
# numbers between 0 and 9 (inclusive),
# and the underscore symbol
  options = string.digits + string.ascii_uppercase + "_"
  h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
  d = h.digest()
  s = ""
  for n in d:
   s += options[n%len(options)]
  return s[0:l]

student_password = get_password('ec23780')
print(student_password)

4PNPF4PW91


In [8]:
# Fitness value Generation for candidates (3.0.2)

def get_password(student_username, l=10):
    # Possible characters include upper-case English letters, numbers between 0 and 9 (inclusive), 
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]

# TO DO: replace *** with your EECS username and uncomment the code
# student_password = get_password('***')
print(student_password)

# Distance function
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


# Upper bound of the distance value
MAX_VALUE = distance_function('0000000000', '__________')

# Compute normalised fitness for a list of candidate passwords 
def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        # Return 1 when a candidate matches the true password (string distance between them is zero)
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE
    return ordered_dict

# Example of how to get fitness values for a list of candidates
get_normalised_fitness(['2Q4HHHHOTJ', '2HHZQYUOTJ'], student_password)

4PNPF4PW91


{'2Q4HHHHOTJ': 0.5320502816987442, '2HHZQYUOTJ': 0.4965551074632897}

In [12]:
# Genetic Algorithm implementation  (3.1.0)

# Distance function
def dist_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


MAX_VALUE = dist_function('0000000000', '__________')

# Random password generation function
def generate_random_password(length=10):
    options = string.digits + string.ascii_uppercase + "_"
    return ''.join(random.choice(options) for _ in range(length))

# Genetic Algorithm Implementation to find the password
def genetic_algorithm(target_password, population_size=100, mutation_rate=0.2, generations=1000):
    population = [generate_random_password() for _ in range(population_size)]

    for generation in range(generations):
        fitness_scores = {password: 1 - distance_function(password, target_password) / MAX_VALUE for password in population}

        # Sorting the population on basis of fitness scores
        sorted_population = sorted(population, key=lambda x: fitness_scores[x], reverse=True)

        
        if sorted_population[0] == target_password:
            return sorted_population[0]

        #  top performers for reproduction
        parents = sorted_population[:int(population_size * 0.2)]

        # Offspring Generation via  crossover and mutation
        offspring = []
        while len(offspring) < population_size - len(parents):
            parent1, parent2 = random.sample(parents, 2)
            crossover_point = random.randint(1, len(parent1) - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            
            # mutation Application
            if random.random() < mutation_rate:
                mutation_point = random.randint(0, len(child) - 1)
                child = child[:mutation_point] + generate_random_password(1) + child[mutation_point + 1:]

            offspring.append(child)

        
        population = parents + offspring

   
    return sorted_population[0]

# actual password retrieved from the fitness function
actual_password = "4PNPF4PW91"

# genetic algorithm to find the password
found_password = genetic_algorithm(actual_password)


print(f"True Password: {actual_password}")
print(f"Found Password: {found_password}")

# testing for different mutation rates as a hyperparameter (3.4) 

# Genetic Algorithm to find the password 
def genetic_algorithm_with_conv(target_password, population_size=100, mutation_rate=0.2):
    population = [generate_random_password() for _ in range(population_size)]
    generations = 0

    while True:
        fitness_scores = {password: 1 - dist_function(password, target_password) / MAX_VALUE for password in population}

        # Sorting the population based on fitness scores
        sorted_population = sorted(population, key=lambda x: fitness_scores[x], reverse=True)

       
        if sorted_population[0] == target_password:
            return generations

        #  the top performers for reproduction
        parents = sorted_population[:int(population_size * 0.2)]

        # offspring Generation through crossover and mutation
        offspring = []
        while len(offspring) < population_size - len(parents):
            parent1, parent2 = random.sample(parents, 2)
            crossover_point = random.randint(1, len(parent1) - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            
            # mutation application
            if random.random() < mutation_rate:
                mutation_point = random.randint(0, len(child) - 1)
                child = child[:mutation_point] + generate_random_password(1) + child[mutation_point + 1:]

            offspring.append(child)

        
        population = parents + offspring
        generations += 1

# actual password retrieved from the fitness function
actual_password = "4PNPF4PW91"

# Genetic algorithm cycles
num_runs = 10
generations_needed = []

for _ in range(num_runs):
    generations_needed.append(genetic_algorithm_with_conv(actual_password))

#  average and standard deviation
avg_generations = sum(generations_needed) / num_runs
std_dev_generations = math.sqrt(sum((gen - avg_generations)**2 for gen in generations_needed) / num_runs)

print(f"Average Generations: {avg_generations}")
print(f"Standard Deviation of Generations: {std_dev_generations}")

True Password: 4PNPF4PW91
Found Password: 4PNPF4PW91
Average Generations: 73.5
Standard Deviation of Generations: 29.196746394076175
